In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [193]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## I. Explore the data  
  
A. Study variable attributes 
 1. ~Identify variable name and survey item(s) it measures (Codebook available here: https://www.worldvaluessurvey.org/WVSDocumentationWV6.jsp (accessed on 5/25/2021))~  
 2. ~% missing for each variable~
 3. Quick descriptives (check for range of values, distribution shape, skew/outliers, potential errors, etc.)  
 4. Identify target variable (and drop duplicates) - don't forget to do feature engineering on the target variable; break out into varying levels of happiness  
  
B. Visualize the data (based on descriptives)
 1. Explore correlations between attributes
 2. Identify transformations that might be needed
 3. Identify extra data that may be useful (gini coefficient, GDP, etc.)
 4. Summarize findings
  
---
  
### Codebook notes:  
  
- Weights (See https://www.worldvaluessurvey.org/WVSContents.jsp for further details): 
    - `S018` and `S019` are weighting factors that transform N's to 1000 and 1500, respectively
    - these variables are useful for cross-country comparisons 
    - useful for EDA and descriptive analyses; should arguably be dropped for random forest algorithm, **right?**
    - **QUESTION:** are weights useful for PCA and logistic regression?
    - **QUESTION:** I see weights, but not specific population or sample size info - do I need this?
        - population data shouldn't be difficult to obtain based on N preserving weightings (`V258`) and this formula  
        
        $$Weight = S018/1000 * Population$$  
  
---
  
### Options for handling missing data:  
Advice from https://heartbeat.fritz.ai/data-handling-scenarios-part-2-working-with-missing-values-in-a-dataset-34b758cfc9fa and https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/  
  
**Mean/Median (numerical) & Mode (categorical) imputation**  
1. pros: 
  - easy to do
  - can be integrated into production or for a future unknown dataset
2. cons: 
  - distorts the distribution of the dataset
  - distorts the variance and covariance of the dataset
  - for mode imputation, may lead to an over-representation of the most frequent label if the missing values are quite large
3. when this makes sense: 
  - mean imputation works best for normally distributed distributions
  - median is better for skewed distributions 
  - mode imputation for categorical data works best if the missing values are missing at random
  - best to use this method when the missing values are around 5% (or less) of the total data
  
**Systematic Random Sampling Imputation**  
1. pros: 
  - does not distort variance or distribution 
2. cons: 
  - when replacing missing values in the test set as well, the imputed values from the train set will need to be stored in memory
3. when this makes sense: 
  - can be applied to both numerical and categorical variables
  - used when the values are missing at random
  - when we want to be able to reproduce the same value every time the variable is used (by using a random state)
    
---  
  
### Thinking ahead to future steps:
- items may need to be normalized or re-scaled so that the ranges are more similar
- ~items may need to be reverse-coded to assist with interpretability for linear regression~
    - not worth the effort at this juncture; revisit codebook when interpreting model output instead
- ~retain and rename `C_COW_ALPHA` for country labels~
- ~recode age variable `V242`; create age categories based on groupings identified here: https://www.cia.gov/the-world-factbook/field/age-structure/~  
- ~`V74` and `V74B`: Schwartz benevolence value items; consolidate into one variable based on whichever has fewer missings~  

**Code for imputing by group using the median**  
df[cols_to_impute] = df[cols_to_impute].fillna(df.groupby('group')[cols_to_impute].transform('median'))  
   
**Code for imputing by group using the mode**  
df[cols_to_impute] = df[cols_to_impute].fillna(df.groupby('group')[cols_to_impute].transform(lambda x: pd.Series.mode(x)[0]))

In [194]:
wvs_w6 = pd.read_csv('../data/Evaluating_Happiness/w6_cleaning.csv', low_memory=False)
wvs_w6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85898 entries, 0 to 85897
Columns: 166 entries, V2 to V262
dtypes: float64(163), int64(2), object(1)
memory usage: 108.8+ MB


In [195]:
wvs_w6.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85898 entries, 0 to 85897
Data columns (total 166 columns):
 #   Column       Dtype  
---  ------       -----  
 0   V2           int64  
 1   C_COW_ALPHA  object 
 2   V4           float64
 3   V5           float64
 4   V6           float64
 5   V7           float64
 6   V8           float64
 7   V9           float64
 8   V10          float64
 9   V11          float64
 10  V12          float64
 11  V13          float64
 12  V14          float64
 13  V15          float64
 14  V16          float64
 15  V17          float64
 16  V18          float64
 17  V19          float64
 18  V20          float64
 19  V21          float64
 20  V22          float64
 21  V24          float64
 22  V25          float64
 23  V26          float64
 24  V27          float64
 25  V30          float64
 26  V32          float64
 27  V33          float64
 28  V34          float64
 29  V44          float64
 30  V45          float64
 31  V47          float64
 32  V

In [196]:
wvs_w6.head()

,V2,C_COW_ALPHA,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V24,V25,V26,V27,V30,V32,V33,V34,V44,V45,V47,V48,V49,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V74B,V75,V76,V77,V78,V79,V80,V82,V83,V84,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V113,V114,V115,V116,V117,V119,V120,V121,V122,V123,V124,V126,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V143,V144G,V147,V150,V151,V152,V153,V154,V155,V170,V171,V173,V174,V176,V177,V179,V180,V181,V182,V183,V184,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V202,V203,V204,V205,V207,V208,V209,V210,V211,V213,V214,V216,V225,V229,V230,V237,V238,V239,V240,V242,V248,V258,S018,S019,V262
0,12,ALG,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,4.0,6.0,0.0,10.0,2.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,8.0,7.0,6.0,8.0,7.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,8.0,5.0,6.0,9.0,3.0,4.0,7.0,6.0,7.0,2.0,5.0,1.0,NaN,NaN,10.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,7.0,8.0,3.0,5.0,6.0,9.0,6.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0,5.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,NaN,1.0,4.0,5.0,1.0,21.0,7.0,1.0,0.833333,1.25,2014
1,12,ALG,1.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,6.0,8.0,6.0,0.0,10.0,2.0,1.0,2.0,3.0,4.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,7.0,5.0,5.0,4.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,8.0,8.0,8.0,9.0,2.0,6.0,4.0,2.0,4.0,1.0,5.0,1.0,2.0,1.0,10.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,4.0,6.0,4.0,8.0,3.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,2.0,3.0,6.0,NaN,2.0,3.0,6.0,2.0,24.0,7.0,1.0,0.833333,1.25,2014
2,12,ALG,1.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,8.0,6.0,0.0,6.0,2.0,4.0,1.0,2.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,5.0,7.0,5.0,1.0,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,7.0,4.0,8.0,3.0,3.0,6.0,9.0,5.0,6.0,1.0,5.0,1.0,2.0,1.0,6.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,3.0,2.0,4.0,2.0,3.0,2.0,1.0,4.0,6.0,2.0,26.0,5.0,1.0,0.833333,1.25,2014
3,12,ALG,1.0,1.0,3.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,8.0,6.0,0.0,6.0,2.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,9.0,5.0,6.0,4.0,6.0,8.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,1.0,2.0,4.0,3.0,2.0,7.0,9.0,5.0,5.0,7.0,3.0,8.0,7.0,8.0,2.0,5.0,1.0,2.0,1.0,10.0,2.0,3.0,4.0,1.0,2.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,6.0,6.0,3.0,5.0,5.0,7.0,4.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,4.0,4.0,5.0,2.0,28.0,6.0,1.0,0.833333,1.25,2014
4,12,ALG,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,1.0,3.0,4.0,2.0,1.0,2.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,2.0,5.0,1.0,2.0,3.0,1.0,4.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,8.0,4.0,7.0,4.0,6.0,6.0,2.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,2.0,8.0,4.0,7.0,3.0,3.0,8.0,6.0,5.0,6.0,2.0,5.0,1.0,1.0,1.0,10.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,6.0,2.0,4.0

In [197]:
# how many unique countries?
wvs_w6.C_COW_ALPHA.nunique()

57

#### Notes along the way:
There are 194 independent countries in the world; this data set contains 57 of them.

In [198]:
# calculate proportion missing for each feature
pct_missing = pd.DataFrame(wvs_w6.isna().sum())
pct_missing = pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
pct_missing['NA_pct'] = ((pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
pct_missing

,variable,NA_count,NA_pct
0,V2,0,0.00
1,C_COW_ALPHA,0,0.00
2,V4,316,0.37
3,V5,502,0.58
4,V6,958,1.12
5,V7,1502,1.75
6,V8,1433,1.67
7,V9,1255,1.46
8,V10,723,0.84
9,V11,302,0.35


In [199]:
wvs_w6.describe()

,V2,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V24,V25,V26,V27,V30,V32,V33,V34,V44,V45,V47,V48,V49,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V74B,V75,V76,V77,V78,V79,V80,V82,V83,V84,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V113,V114,V115,V116,V117,V119,V120,V121,V122,V123,V124,V126,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V143,V144G,V147,V150,V151,V152,V153,V154,V155,V170,V171,V173,V174,V176,V177,V179,V180,V181,V182,V183,V184,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V202,V203,V204,V205,V207,V208,V209,V210,V211,V213,V214,V216,V225,V229,V230,V237,V238,V239,V240,V242,V248,V258,S018,S019,V262
count,85898.000000,85582.000000,85396.000000,84940.000000,84396.000000,84465.000000,84643.000000,85175.000000,85596.000000,85884.000000,85892.000000,85891.000000,85881.000000,85889.000000,85885.000000,85886.000000,85880.000000,85883.00000,85879.000000,85895.000000,83595.000000,85293.000000,85233.000000,85183.000000,85090.000000,85107.000000,85008.000000,85058.000000,85878.000000,84325.000000,82573.000000,83057.000000,82824.000000,81947.000000,83370.000000,82125.000000,81469.000000,84510.000000,84076.000000,85693.000000,84259.000000,85328.000000,84201.000000,81915.000000,84505.000000,82634.000000,84619.000000,82927.000000,77327.000000,82722.000000,82512.000000,82239.000000,82990.00000,83366.000000,83863.000000,83477.000000,80373.000000,37101.000000,83133.000000,82837.000000,83635.000000,83650.000000,83853.000000,84865.000000,84930.000000,84646.000000,84952.000000,83105.000000,80722.000000,83717.000000,82810.000000,83706.000000,81419.000000,85307.000000,84733.000000,84826.000000,83656.000000,79882.000000,79646.000000,83122.000000,83155.000000,83244.000000,84002.000000,83922.000000,82685.000000,82854.000000,80943.000000,81580.000000,80733.000000,79679.000000,82030.000000,78682.000000,77130.000000,79284.000000,75354.000000,82219.000000,80256.000000,82694.000000,82662.000000,80273.000000,80824.000000,81885.000000,81336.000000,83154.000000,83737.000000,84290.000000,84646.000000,82980.000000,80608.000000,78748.000000,83872.000000,77374.000000,77818.000000,78000.000000,84123.000000,81831.000000,78908.000000,76478.000000,83956.000000,84185.000000,84559.000000,83286.000000,80662.000000,79188.000000,82809.000000,82818.000000,80446.000000,84643.000000,84201.000000,84320.000000,84342.000000,83066.000000,82790.000000,81240.000000,79483.000000,82395.000000,83226.000000,83527.000000,84236.000000,84582.000000,84266.000000,81432.000000,82875.00000,83437.000000,83329.000000,84483.000000,84504.000000,84466.000000,84460.000000,83405.000000,84393.000000,81173.000000,84717.000000,84419.000000,64985.000000,79979.000000,83479.000000,82875.000000,85858.000000,85789.000000,85173.000000,85898.000000,85898.000000,85898.000000,85898.000000
mean,465.906983,1.096107,1.676847,1.881976,2.639319,1.510057,1.905769,1.836924,2.096184,1.489334,1.390944,1.282335,1.768983,1.317677,1.601816,1.599946,1.599965,1.65956,1.584147,1.730625,1.756158,0.522751,0.327948,0.238240,0.131308,0.189385,0.111331,0.165181,1.831598,2.043724,2.097175,1.604308,1.658662,2.439626,2.946408,2.569851,2.220329,7.064998,5.675615,2.735054,1.852965,5.872070,1.706049,2.562559,2.052742,2.496769,1.879093,2.737504,1.330829,2.289850,1.380369,1.507022,2.78201,3.773541,2.382016,3.208800,2.465816,2.378966,2.915016,3.727477,2.551085,2.540167,2.545788,2.162175,1.877629,1.930581,2.657053,5.281174,5.586432,4.464637,3.876054,4.229518,6.216055,1.212480,2.147192,2.074788,3.076372,2.763101,2.840231,2.149635,2.188010,2.606590,2.508952,2.421439,2.440406,2.591498,2.927888,2.731723,2.140996,2.512130,2.416360,2.372970,2.375172,2.335465,2.638626,6.301950,4.201505,8.004583,7.014178,4.499682,7.398607,5.976064,5.972657,7.899969,8.240646,1.833764,3.514224,1.364256,1.719631,1.722253,7.777244,2.413020,2.473091,2.568410,1.889459,3.160550,3.470992,3.543817,2.645243,3.003682,4.682872,4.579857

### Remaining feature engineering/data prep steps:
- ~rename `C_COW_ALPHA`~
- ~recode age variable `V242` into groups~ 
    - 0-14 years (children) = _[excluded from analysis]_
    - 15-24 years (early working age) = _[group 1]_
    - 25-54 years (prime working age) = _[group 2]_
    - 55-64 years (mature working age) = _[group 3]_
    - 65 years and over (elderly) = _[group 4]_
    - **note:** ended up having to do this after imputing missing values
- ~consolidate Schwartz variables `V74` and `V74B`~

In [200]:
# how many non-missing age values?
wvs_w6.V242.notna().sum()

85789

In [201]:
# how many missing values for v242?
wvs_w6.V242.isna().sum()

109

### Rename `C_COW_ALPHA`

In [202]:
# make all column names lowercase
wvs_w6.columns = wvs_w6.columns.str.lower()

# rename country column
wvs_w6.rename(columns = {'c_cow_alpha': 'country'},
              inplace = True)
wvs_w6.head()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v74b,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v230,v237,v238,v239,v240,v242,v248,v258,s018,s019,v262
0,12,ALG,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,4.0,6.0,0.0,10.0,2.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,8.0,7.0,6.0,8.0,7.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,8.0,5.0,6.0,9.0,3.0,4.0,7.0,6.0,7.0,2.0,5.0,1.0,NaN,NaN,10.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,7.0,8.0,3.0,5.0,6.0,9.0,6.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0,5.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,NaN,1.0,4.0,5.0,1.0,21.0,7.0,1.0,0.833333,1.25,2014
1,12,ALG,1.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,6.0,8.0,6.0,0.0,10.0,2.0,1.0,2.0,3.0,4.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,7.0,5.0,5.0,4.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,8.0,8.0,8.0,9.0,2.0,6.0,4.0,2.0,4.0,1.0,5.0,1.0,2.0,1.0,10.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,4.0,6.0,4.0,8.0,3.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,2.0,3.0,6.0,NaN,2.0,3.0,6.0,2.0,24.0,7.0,1.0,0.833333,1.25,2014
2,12,ALG,1.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,8.0,6.0,0.0,6.0,2.0,4.0,1.0,2.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,5.0,7.0,5.0,1.0,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,7.0,4.0,8.0,3.0,3.0,6.0,9.0,5.0,6.0,1.0,5.0,1.0,2.0,1.0,6.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,3.0,2.0,4.0,2.0,3.0,2.0,1.0,4.0,6.0,2.0,26.0,5.0,1.0,0.833333,1.25,2014
3,12,ALG,1.0,1.0,3.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,8.0,6.0,0.0,6.0,2.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,9.0,5.0,6.0,4.0,6.0,8.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,1.0,2.0,4.0,3.0,2.0,7.0,9.0,5.0,5.0,7.0,3.0,8.0,7.0,8.0,2.0,5.0,1.0,2.0,1.0,10.0,2.0,3.0,4.0,1.0,2.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,6.0,6.0,3.0,5.0,5.0,7.0,4.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,4.0,4.0,5.0,2.0,28.0,6.0,1.0,0.833333,1.25,2014
4,12,ALG,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,1.0,3.0,4.0,2.0,1.0,2.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,2.0,5.0,1.0,2.0,3.0,1.0,4.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,8.0,4.0,7.0,4.0,6.0,6.0,2.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,2.0,8.0,4.0,7.0,3.0,3.0,8.0,6.0,5.0,6.0,2.0,5.0,1.0,1.0,1.0,10.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,6.0,2.0,4.0,4.0

### Consolidate Schwartz `v74` and `v74b` variables

In [203]:
# consolidate v74 and v74b
wvs_w6[['country', 'v74', 'v74b']].sample()

,country,v74,v74b
8498,BRA,2.0,2.0


In [204]:
schw_bene = wvs_w6[['country', 'v74', 'v74b']]
schw_bene.sample(10)

,country,v74,v74b
40301,LIB,1.0,1.0
35019,KZK,1.0,NaN
30668,JPN,NaN,NaN
65236,SAF,1.0,1.0
83338,UZB,3.0,NaN
84363,YEM,3.0,2.0
22265,HAI,5.0,2.0
35239,KZK,3.0,NaN
38103,LEB,3.0,2.0
63565,SLV,2.0,NaN


In [205]:
# add columns indicating whether respondent was more likely to relate more to helping society, helping neighbors,
# or helping both equally
schw_bene['help_neighbor'] = schw_bene['v74'] > schw_bene['v74b']
schw_bene['help_society'] = schw_bene['v74b'] > schw_bene['v74']
schw_bene['help_equally'] = schw_bene['v74'] == schw_bene['v74b']

<ipython-input-205-e528dde403fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schw_bene['help_neighbor'] = schw_bene['v74'] > schw_bene['v74b']
<ipython-input-205-e528dde403fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schw_bene['help_society'] = schw_bene['v74b'] > schw_bene['v74']
<ipython-input-205-e528dde403fc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [206]:
schw_bene.sample(20)

,country,v74,v74b,help_neighbor,help_society,help_equally
45319,NTH,2.0,2.0,False,False,True
38058,LEB,2.0,1.0,True,False,False
60393,RWA,2.0,4.0,False,True,False
81670,URU,3.0,NaN,False,False,False
41654,MAL,4.0,NaN,False,False,False
73915,TRI,3.0,NaN,False,False,False
32574,JPN,3.0,3.0,False,False,True
29213,IND,1.0,NaN,False,False,False
69942,SWD,4.0,NaN,False,False,False
79884,USA,4.0,NaN,False,False,False


In [207]:
# count up total True values for each column by country to get a quick view of patterns by country
schw_bene['missing_entirely'] = schw_bene.v74.isna() & schw_bene.v74b.isna()

<ipython-input-207-3c99083b9c93>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schw_bene['missing_entirely'] = schw_bene.v74.isna() & schw_bene.v74b.isna()


In [208]:
schw_bene.sample(50)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely
32657,JPN,1.0,1.0,False,False,True,False
59489,RWA,3.0,3.0,False,False,True,False
33174,JOR,2.0,2.0,False,False,True,False
17410,GRG,1.0,1.0,False,False,True,False
47904,NIG,3.0,NaN,False,False,False,False
60300,RWA,3.0,2.0,True,False,False,False
45803,NTH,3.0,3.0,False,False,True,False
70771,SWD,3.0,NaN,False,False,False,False
16076,EST,4.0,NaN,False,False,False,False
35066,KZK,4.0,NaN,False,False,False,False


In [209]:
# how many cases (regardless of country) have missing data in both columns?
schw_bene.missing_entirely.sum()

1931

In [210]:
# total cases
schw_bene.missing_entirely.count()

85898

In [211]:
# how many cases by country have missing data in both columns?
schw_bene.groupby(['country']).sum()

,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely
country,,,,,,
ALG,2786.0,2728.0,276.0,237.0,595.0,60.0
ARG,2594.0,0.0,0.0,0.0,0.0,16.0
ARM,2454.0,0.0,0.0,0.0,0.0,3.0
AUL,3928.0,0.0,0.0,0.0,0.0,29.0
AZE,2888.0,0.0,0.0,0.0,0.0,0.0
BLR,4506.0,0.0,0.0,0.0,0.0,15.0
BRA,2910.0,2652.0,381.0,168.0,932.0,3.0
CHL,1955.0,0.0,0.0,0.0,0.0,75.0
CHN,5846.0,5402.0,525.0,226.0,1416.0,125.0


In [212]:
schw_groupby = schw_bene.groupby(['country']).sum()
schw_groupby.reset_index()
schw_groupby

,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely
country,,,,,,
ALG,2786.0,2728.0,276.0,237.0,595.0,60.0
ARG,2594.0,0.0,0.0,0.0,0.0,16.0
ARM,2454.0,0.0,0.0,0.0,0.0,3.0
AUL,3928.0,0.0,0.0,0.0,0.0,29.0
AZE,2888.0,0.0,0.0,0.0,0.0,0.0
BLR,4506.0,0.0,0.0,0.0,0.0,15.0
BRA,2910.0,2652.0,381.0,168.0,932.0,3.0
CHL,1955.0,0.0,0.0,0.0,0.0,75.0
CHN,5846.0,5402.0,525.0,226.0,1416.0,125.0


In [213]:
# countries whose respondents valued helping neighbor and helping society (v74 = v74b) most often
schw_groupby.loc[(schw_groupby['help_equally'] > schw_groupby['help_neighbor']) & \
                 (schw_groupby['help_equally'] > schw_groupby['help_society'])]

,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely
country,,,,,,
ALG,2786.0,2728.0,276.0,237.0,595.0,60.0
BRA,2910.0,2652.0,381.0,168.0,932.0,3.0
CHN,5846.0,5402.0,525.0,226.0,1416.0,125.0
COL,2812.0,2784.0,269.0,284.0,955.0,1.0
ECU,2669.0,2884.0,238.0,396.0,566.0,1.0
GRG,2414.0,2191.0,286.0,110.0,793.0,3.0
IRQ,2604.0,2541.0,376.0,326.0,485.0,1.0
JOR,2395.0,2197.0,322.0,199.0,672.0,3.0
JPN,8407.0,7766.0,833.0,231.0,1092.0,147.0


In [214]:
help_equal_df = pd.DataFrame(schw_groupby.loc[(schw_groupby['help_equally'] > schw_groupby['help_neighbor']) & \
                 (schw_groupby['help_equally'] > schw_groupby['help_society'])])
help_equal_list = help_equal_df.reset_index().country.tolist()
print(help_equal_list)
print(len(help_equal_list))

['ALG', 'BRA', 'CHN', 'COL', 'ECU', 'GRG', 'IRQ', 'JOR', 'JPN', 'LEB', 'LIB', 'NTH', 'PAK', 'PSE', 'RWA', 'SAF', 'SIN', 'THI', 'TUN', 'YEM']
20


In [215]:
# countries whose respondents valued helping neighbor over helping society (v74 > v74b) most often
help_neighbor_df = pd.DataFrame(schw_groupby.loc[(schw_groupby['help_neighbor'] > schw_groupby['help_society'])])
help_neighbor_list = help_neighbor_df.reset_index().country.tolist()
print(help_neighbor_list)
print(len(help_neighbor_list))

['ALG', 'BRA', 'CHN', 'GRG', 'HKG', 'IRQ', 'JOR', 'JPN', 'LEB', 'LIB', 'NTH', 'PSE', 'RWA', 'SAF', 'TUN', 'YEM']
16


In [216]:
# countries whose respondents valued helping society over helping neighbor (v74b > v74) most often
help_society_df = pd.DataFrame(schw_groupby.loc[(schw_groupby['help_society'] > schw_groupby['help_neighbor'])])
help_society_list = help_society_df.reset_index().country.tolist()
print(help_society_list)
print(len(help_society_list))

['COL', 'ECU', 'HAI', 'PAK', 'SIN', 'THI']
6


In [217]:
len(schw_groupby)

57

#### Approach to missings for v74 and v74b should vary by situation:  
  
**For 20 countries in help_equal_list:**  
I am using trends in the data to assume that for the most part, people tend to answer items v74 and v74b in the same way. To handle missing values for these countries:
- step 1: fill v74 or v74b NaN values with values from the other column if those data are available
- step 2: if data are missing for v74 AND v74b columns, proceed to options below

**For 16 countries in help_neighbor_list:**  
For the most part, people tended to show a willingness to help close neighbors more often than help society  
- impute median of v74b (although this skews the distribution of responses)  
  
**For 6 countries in help_society_list:**  
For the most part, people tended to show a willingness to help help society more often than help close neighbors  
- impute median of v74 (although this skews the distribution of responses)  
  
**Next Steps:** replace v74 and v74b with a combination of the two to reduce and deal with missing values in both  
- 6.43% of values are missing for v74, but 56.81% of values are missing for v74b  
- v74 will be the default source of values for the replacement variable

In [218]:
# create new variable for 'v74' and 'v74b' called 'v74r' ('r' for replacement) based on 'v74' values
schw_bene['v74r'] = schw_bene['v74']

<ipython-input-218-91d7d616b334>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schw_bene['v74r'] = schw_bene['v74']


In [219]:
schw_bene.sample(10)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
67530,SAF,2.0,1.0,True,False,False,False,2.0
80487,USA,1.0,NaN,False,False,False,False,1.0
79191,USA,3.0,NaN,False,False,False,False,3.0
44749,MOR,NaN,2.0,False,False,False,False,NaN
5536,AZE,1.0,NaN,False,False,False,False,1.0
19165,GMY,3.0,NaN,False,False,False,False,3.0
14153,CHN,3.0,3.0,False,False,True,False,3.0
68518,SAF,1.0,3.0,False,True,False,False,1.0
35151,KZK,3.0,NaN,False,False,False,False,3.0
83192,UZB,1.0,NaN,False,False,False,False,1.0


In [220]:
# how many missings in new variable (should be same as missings count for v74)
schw_bene.v74r.isna().sum()

5525

In [221]:
# impute missings using v74b for countries in list where responses to v74 and v74b tend to be the same
# for countries that are in help_equal_list, fill missing values in v74r with values from v74b, otherwise just use existing
# v74r values from the previous step
schw_bene.v74r = np.where(schw_bene.country.isin(help_equal_list), schw_bene.v74r.fillna(schw_bene.v74b), schw_bene.v74r)

C:\Users\ocnra\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [222]:
schw_bene.sample(10)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
71032,TAW,1.0,NaN,False,False,False,False,1.0
28909,IND,5.0,NaN,False,False,False,False,5.0
12679,CHN,3.0,2.0,True,False,False,False,3.0
36420,KYR,2.0,NaN,False,False,False,False,2.0
28741,IND,1.0,NaN,False,False,False,False,1.0
85432,ZIM,4.0,NaN,False,False,False,False,4.0
47909,NIG,1.0,NaN,False,False,False,False,1.0
80088,USA,3.0,NaN,False,False,False,False,3.0
77643,UKR,2.0,NaN,False,False,False,False,2.0
38619,LIB,4.0,4.0,False,False,True,False,4.0


In [223]:
schw_bene[schw_bene['country'] == 'ALG'].sample(10)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
780,ALG,1.0,3.0,False,True,False,False,1.0
532,ALG,6.0,6.0,False,False,True,False,6.0
555,ALG,NaN,NaN,False,False,False,True,NaN
158,ALG,2.0,2.0,False,False,True,False,2.0
395,ALG,2.0,2.0,False,False,True,False,2.0
688,ALG,NaN,1.0,False,False,False,False,1.0
323,ALG,3.0,3.0,False,False,True,False,3.0
424,ALG,3.0,1.0,True,False,False,False,3.0
386,ALG,3.0,1.0,True,False,False,False,3.0
458,ALG,2.0,3.0,False,True,False,False,2.0


In [224]:
# compare distributions so far
schw_bene[schw_bene['country'] == 'ALG'][['v74', 'v74b', 'v74r']].describe()

,v74,v74b,v74r
count,1112.000000,1136.000000,1140.000000
mean,2.505396,2.401408,2.482456
std,1.523878,1.432461,1.517428
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000
75%,3.000000,3.000000,3.000000
max,6.000000,6.000000,6.000000


In [225]:
# impute missings using v74b for countries that tend to value helping neighbors more
schw_bene.v74r = np.where(schw_bene.country.isin(help_neighbor_list), \
                          schw_bene.v74r.fillna(schw_bene.groupby('country')['v74b'].transform('median')), \
                          schw_bene.v74r)

C:\Users\ocnra\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [226]:
# check for a country in the help neighbors more list
schw_bene[schw_bene['country'] == 'ALG'].sample(30)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
206,ALG,1.0,2.0,False,True,False,False,1.0
762,ALG,2.0,4.0,False,True,False,False,2.0
676,ALG,1.0,3.0,False,True,False,False,1.0
1068,ALG,2.0,3.0,False,True,False,False,2.0
745,ALG,2.0,1.0,True,False,False,False,2.0
5,ALG,5.0,4.0,True,False,False,False,5.0
224,ALG,NaN,NaN,False,False,False,True,2.0
701,ALG,2.0,2.0,False,False,True,False,2.0
1036,ALG,1.0,1.0,False,False,True,False,1.0
1070,ALG,3.0,2.0,True,False,False,False,3.0


In [227]:
# check for a country in the help society more list
schw_bene[schw_bene['country'] == 'RUS'].sample(30)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
59194,RUS,3.0,NaN,False,False,False,False,3.0
56749,RUS,3.0,NaN,False,False,False,False,3.0
57990,RUS,4.0,NaN,False,False,False,False,4.0
58210,RUS,2.0,NaN,False,False,False,False,2.0
56885,RUS,2.0,NaN,False,False,False,False,2.0
57518,RUS,3.0,NaN,False,False,False,False,3.0
57020,RUS,2.0,NaN,False,False,False,False,2.0
58236,RUS,2.0,NaN,False,False,False,False,2.0
58306,RUS,3.0,NaN,False,False,False,False,3.0
57286,RUS,2.0,NaN,False,False,False,False,2.0


In [228]:
# check for a country in the help society more list
schw_bene[schw_bene['country'] == 'IND'].sample(30)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
25687,IND,1.0,NaN,False,False,False,False,1.0
28249,IND,1.0,NaN,False,False,False,False,1.0
26597,IND,1.0,NaN,False,False,False,False,1.0
28357,IND,1.0,NaN,False,False,False,False,1.0
26826,IND,NaN,NaN,False,False,False,True,NaN
26654,IND,NaN,NaN,False,False,False,True,NaN
28505,IND,3.0,NaN,False,False,False,False,3.0
27345,IND,5.0,NaN,False,False,False,False,5.0
25523,IND,3.0,NaN,False,False,False,False,3.0
28160,IND,1.0,NaN,False,False,False,False,1.0


In [229]:
# compare distributions again
schw_bene[schw_bene['country'] == 'ALG'][['v74', 'v74b', 'v74r']].describe()

,v74,v74b,v74r
count,1112.000000,1136.000000,1200.000000
mean,2.505396,2.401408,2.458333
std,1.523878,1.432461,1.482710
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000
75%,3.000000,3.000000,3.000000
max,6.000000,6.000000,6.000000


In [230]:
# impute missings using v74b for countries that tend to value helping society more
schw_bene.v74r = np.where(schw_bene.country.isin(help_society_list), \
                          schw_bene.v74r.fillna(schw_bene.groupby('country')['v74'].transform('median')), \
                          schw_bene.v74r)

C:\Users\ocnra\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [231]:
# check row previously containing a missing value from respondent in Singapore
schw_bene[(schw_bene['country'] == 'SIN') & (schw_bene['missing_entirely'] == True)]

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
61744,SIN,NaN,NaN,False,False,False,True,3.0


In [232]:
schw_bene[schw_bene['country'] == 'SIN'][['v74', 'v74b', 'v74r']].describe()

,v74,v74b,v74r
count,1971.000000,1971.000000,1972.000000
mean,2.674277,2.676306,2.674442
std,1.118122,1.110056,1.117862
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,3.000000,3.000000,3.000000
75%,3.000000,3.000000,3.000000
max,6.000000,6.000000,6.000000


In [233]:
# how many missing values in v74r compared to v74 and v74b?
schw_bene.isna().sum()

country                 0
v74                  5525
v74b                48797
help_neighbor           0
help_society            0
help_equally            0
missing_entirely        0
v74r                 4542
dtype: int64

In [234]:
# which rows still have missing data across the board?
schw_bene[schw_bene['v74r'].isnull()]

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
1246,ARG,NaN,NaN,False,False,False,True,NaN
1285,ARG,NaN,NaN,False,False,False,True,NaN
1357,ARG,NaN,NaN,False,False,False,True,NaN
1379,ARG,NaN,NaN,False,False,False,True,NaN
1386,ARG,NaN,NaN,False,False,False,True,NaN
1453,ARG,NaN,NaN,False,False,False,True,NaN
1578,ARG,NaN,NaN,False,False,False,True,NaN
1951,ARG,NaN,NaN,False,False,False,True,NaN
2036,ARG,NaN,NaN,False,False,False,True,NaN
2038,ARG,NaN,NaN,False,False,False,True,NaN


In [235]:
# which countries are involved?
schw_bene[schw_bene['v74r'].isnull()].country.unique()

array(['ARG', 'ARM', 'AUL', 'BLR', 'CYP', 'CHL', 'EST', 'GMY', 'IND',
       'KYR', 'MEX', 'MOR', 'PER', 'PHI', 'POL', 'ROM', 'RUS', 'SLV',
       'ROK', 'SPN', 'SWD', 'TAW', 'TRI', 'TUR', 'USA', 'URU', 'UZB'],
      dtype=object)

**Based on tendencies observed earlier,**  
  
Impute missings using v74b:  
- 'MOR'
- 'SPN'  
  
Impute missings using v74:  
- all other countries  

In [236]:
# create a list of countries with remaining missings for v74r
v74_only_list = schw_bene[schw_bene['v74r'].isnull()].country.unique().tolist()

In [237]:
# drop Morroco and Spain from list
v74_only_list.remove('MOR')
v74_only_list.remove('SPN')

In [238]:
v74_only_list

['ARG',
 'ARM',
 'AUL',
 'BLR',
 'CYP',
 'CHL',
 'EST',
 'GMY',
 'IND',
 'KYR',
 'MEX',
 'PER',
 'PHI',
 'POL',
 'ROM',
 'RUS',
 'SLV',
 'ROK',
 'SWD',
 'TAW',
 'TRI',
 'TUR',
 'USA',
 'URU',
 'UZB']

In [239]:
# impute missings using v74b for MOR and SPN where respondents only received item v74b
schw_bene.v74r = np.where(schw_bene.country.isin(['MOR', 'SPN']), \
                          schw_bene.v74r.fillna(schw_bene.groupby('country')['v74b'].transform('median')), \
                          schw_bene.v74r).round(2)

C:\Users\ocnra\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [240]:
# impute missings using v74 for countries where respondents only received item v74
schw_bene.v74r = np.where(schw_bene.country.isin(v74_only_list), \
                          schw_bene.v74r.fillna(schw_bene.groupby('country')['v74'].transform('median')), \
                          schw_bene.v74r).round(2)

In [241]:
# how many missing values in v74r compared to v74 and v74b?
schw_bene.isna().sum()

country                 0
v74                  5525
v74b                48797
help_neighbor           0
help_society            0
help_equally            0
missing_entirely        0
v74r                    0
dtype: int64

In [242]:
# check to see if missing values were imputed correctly
schw_bene[schw_bene['v74'].isnull()].sample(7, random_state = 321)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
69305,SPN,NaN,5.0,False,False,False,False,2.0
18663,GMY,NaN,1.0,False,False,False,False,3.0
18634,GMY,NaN,3.0,False,False,False,False,3.0
69059,SPN,NaN,3.0,False,False,False,False,2.0
19085,GMY,NaN,3.0,False,False,False,False,3.0
69961,SWD,NaN,NaN,False,False,False,True,3.0
46199,NTH,NaN,2.0,False,False,False,False,2.0


In [243]:
# overview of created v74r variable
schw_bene.v74r.describe()

count    85898.000000
mean         2.464691
std          1.221127
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: v74r, dtype: float64

In [244]:
# overview of original v74 variable on which v74r is based
schw_bene.v74.describe()

count    80373.000000
mean         2.465816
std          1.251423
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: v74, dtype: float64

In [245]:
schw_bene[schw_bene['v74b'].isnull()].sample(7, random_state = 322)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
28500,IND,NaN,NaN,False,False,False,True,3.0
47322,NIG,1.0,NaN,False,False,False,False,1.0
71111,TAW,3.0,NaN,False,False,False,False,3.0
25753,IND,3.0,NaN,False,False,False,False,3.0
11756,CHL,3.0,NaN,False,False,False,False,3.0
82134,UZB,2.0,NaN,False,False,False,False,2.0
32506,JPN,NaN,NaN,False,False,False,True,4.0


In [246]:
schw_bene[schw_bene['missing_entirely'] == True].sample(7, random_state = 312)

,country,v74,v74b,help_neighbor,help_society,help_equally,missing_entirely,v74r
56784,RUS,NaN,NaN,False,False,False,True,3.0
28568,IND,NaN,NaN,False,False,False,True,3.0
25,ALG,NaN,NaN,False,False,False,True,2.0
78803,USA,NaN,NaN,False,False,False,True,3.0
28872,IND,NaN,NaN,False,False,False,True,3.0
28370,IND,NaN,NaN,False,False,False,True,3.0
71427,TAW,NaN,NaN,False,False,False,True,3.0


In [247]:
wvs_w6.v74.describe()

count    80373.000000
mean         2.465816
std          1.251423
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: v74, dtype: float64

In [248]:
# replace v74 values in wvs_w6 with v74r values in schw_bene
wvs_w6['v74'] = schw_bene['v74r']

In [249]:
wvs_w6.v74.describe()

count    85898.000000
mean         2.464691
std          1.221127
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: v74, dtype: float64

In [250]:
# rename v74 to keep track of this substitution
wvs_w6.rename(columns={'v74':'v74r'}, inplace=True)
wvs_w6.head()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v74b,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v230,v237,v238,v239,v240,v242,v248,v258,s018,s019,v262
0,12,ALG,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,4.0,6.0,0.0,10.0,2.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,8.0,7.0,6.0,8.0,7.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,8.0,5.0,6.0,9.0,3.0,4.0,7.0,6.0,7.0,2.0,5.0,1.0,NaN,NaN,10.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,7.0,8.0,3.0,5.0,6.0,9.0,6.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0,5.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,NaN,1.0,4.0,5.0,1.0,21.0,7.0,1.0,0.833333,1.25,2014
1,12,ALG,1.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,6.0,8.0,6.0,0.0,10.0,2.0,1.0,2.0,3.0,4.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,7.0,5.0,5.0,4.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,8.0,8.0,8.0,9.0,2.0,6.0,4.0,2.0,4.0,1.0,5.0,1.0,2.0,1.0,10.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,4.0,6.0,4.0,8.0,3.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,2.0,3.0,6.0,NaN,2.0,3.0,6.0,2.0,24.0,7.0,1.0,0.833333,1.25,2014
2,12,ALG,1.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,8.0,6.0,0.0,6.0,2.0,4.0,1.0,2.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,3.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,5.0,7.0,5.0,1.0,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,7.0,4.0,8.0,3.0,3.0,6.0,9.0,5.0,6.0,1.0,5.0,1.0,2.0,1.0,6.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,3.0,2.0,4.0,2.0,3.0,2.0,1.0,4.0,6.0,2.0,26.0,5.0,1.0,0.833333,1.25,2014
3,12,ALG,1.0,1.0,3.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,8.0,6.0,0.0,6.0,2.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,9.0,5.0,6.0,4.0,6.0,8.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,1.0,2.0,4.0,3.0,2.0,7.0,9.0,5.0,5.0,7.0,3.0,8.0,7.0,8.0,2.0,5.0,1.0,2.0,1.0,10.0,2.0,3.0,4.0,1.0,2.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,6.0,6.0,3.0,5.0,5.0,7.0,4.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,4.0,4.0,5.0,2.0,28.0,6.0,1.0,0.833333,1.25,2014
4,12,ALG,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,1.0,3.0,4.0,2.0,1.0,2.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,2.0,5.0,1.0,2.0,3.0,1.0,4.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,8.0,4.0,7.0,4.0,6.0,6.0,2.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,2.0,8.0,4.0,7.0,3.0,3.0,8.0,6.0,5.0,6.0,2.0,5.0,1.0,1.0,1.0,10.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,6.0,2.0,4.0,4.

In [251]:
# drop v74b
wvs_w6.drop(['v74b'], axis=1, inplace=True)
wvs_w6.head()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v230,v237,v238,v239,v240,v242,v248,v258,s018,s019,v262
0,12,ALG,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,4.0,6.0,0.0,10.0,2.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,8.0,7.0,6.0,8.0,7.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,8.0,5.0,6.0,9.0,3.0,4.0,7.0,6.0,7.0,2.0,5.0,1.0,NaN,NaN,10.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,7.0,8.0,3.0,5.0,6.0,9.0,6.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0,5.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,NaN,1.0,4.0,5.0,1.0,21.0,7.0,1.0,0.833333,1.25,2014
1,12,ALG,1.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,6.0,8.0,6.0,0.0,10.0,2.0,1.0,2.0,3.0,4.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,7.0,5.0,5.0,4.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,8.0,8.0,8.0,9.0,2.0,6.0,4.0,2.0,4.0,1.0,5.0,1.0,2.0,1.0,10.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,4.0,6.0,4.0,8.0,3.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,2.0,3.0,6.0,NaN,2.0,3.0,6.0,2.0,24.0,7.0,1.0,0.833333,1.25,2014
2,12,ALG,1.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,8.0,6.0,0.0,6.0,2.0,4.0,1.0,2.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,5.0,7.0,5.0,1.0,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,7.0,4.0,8.0,3.0,3.0,6.0,9.0,5.0,6.0,1.0,5.0,1.0,2.0,1.0,6.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,3.0,2.0,4.0,2.0,3.0,2.0,1.0,4.0,6.0,2.0,26.0,5.0,1.0,0.833333,1.25,2014
3,12,ALG,1.0,1.0,3.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,8.0,6.0,0.0,6.0,2.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,9.0,5.0,6.0,4.0,6.0,8.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,1.0,2.0,4.0,3.0,2.0,7.0,9.0,5.0,5.0,7.0,3.0,8.0,7.0,8.0,2.0,5.0,1.0,2.0,1.0,10.0,2.0,3.0,4.0,1.0,2.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,6.0,6.0,3.0,5.0,5.0,7.0,4.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,4.0,4.0,5.0,2.0,28.0,6.0,1.0,0.833333,1.25,2014
4,12,ALG,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,1.0,3.0,4.0,2.0,1.0,2.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,2.0,5.0,1.0,2.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,8.0,4.0,7.0,4.0,6.0,6.0,2.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,2.0,8.0,4.0,7.0,3.0,3.0,8.0,6.0,5.0,6.0,2.0,5.0,1.0,1.0,1.0,10.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,6.0,2.0,4.0,4.0,6.0,6.0,6.0,5.0,7.0,1.0

In [252]:
wvs_w6.describe()

,v2,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v230,v237,v238,v239,v240,v242,v248,v258,s018,s019,v262
count,85898.000000,85582.000000,85396.000000,84940.000000,84396.000000,84465.000000,84643.000000,85175.000000,85596.000000,85884.000000,85892.000000,85891.000000,85881.000000,85889.000000,85885.000000,85886.000000,85880.000000,85883.00000,85879.000000,85895.000000,83595.000000,85293.000000,85233.000000,85183.000000,85090.000000,85107.000000,85008.000000,85058.000000,85878.000000,84325.000000,82573.000000,83057.000000,82824.000000,81947.000000,83370.000000,82125.000000,81469.000000,84510.000000,84076.000000,85693.000000,84259.000000,85328.000000,84201.000000,81915.000000,84505.000000,82634.000000,84619.000000,82927.000000,77327.000000,82722.000000,82512.000000,82239.000000,82990.00000,83366.000000,83863.000000,83477.000000,85898.000000,83133.000000,82837.000000,83635.000000,83650.000000,83853.000000,84865.000000,84930.000000,84646.000000,84952.000000,83105.000000,80722.000000,83717.000000,82810.000000,83706.000000,81419.000000,85307.000000,84733.000000,84826.000000,83656.000000,79882.000000,79646.000000,83122.000000,83155.000000,83244.000000,84002.000000,83922.000000,82685.000000,82854.000000,80943.000000,81580.000000,80733.000000,79679.000000,82030.000000,78682.000000,77130.000000,79284.000000,75354.000000,82219.000000,80256.000000,82694.000000,82662.000000,80273.000000,80824.000000,81885.000000,81336.000000,83154.000000,83737.000000,84290.000000,84646.000000,82980.000000,80608.000000,78748.000000,83872.000000,77374.000000,77818.000000,78000.000000,84123.000000,81831.000000,78908.000000,76478.000000,83956.000000,84185.000000,84559.000000,83286.000000,80662.000000,79188.000000,82809.000000,82818.000000,80446.000000,84643.000000,84201.000000,84320.000000,84342.000000,83066.000000,82790.000000,81240.000000,79483.000000,82395.000000,83226.000000,83527.000000,84236.000000,84582.000000,84266.000000,81432.000000,82875.00000,83437.000000,83329.000000,84483.000000,84504.000000,84466.000000,84460.000000,83405.000000,84393.000000,81173.000000,84717.000000,84419.000000,64985.000000,79979.000000,83479.000000,82875.000000,85858.000000,85789.000000,85173.000000,85898.000000,85898.000000,85898.000000,85898.000000
mean,465.906983,1.096107,1.676847,1.881976,2.639319,1.510057,1.905769,1.836924,2.096184,1.489334,1.390944,1.282335,1.768983,1.317677,1.601816,1.599946,1.599965,1.65956,1.584147,1.730625,1.756158,0.522751,0.327948,0.238240,0.131308,0.189385,0.111331,0.165181,1.831598,2.043724,2.097175,1.604308,1.658662,2.439626,2.946408,2.569851,2.220329,7.064998,5.675615,2.735054,1.852965,5.872070,1.706049,2.562559,2.052742,2.496769,1.879093,2.737504,1.330829,2.289850,1.380369,1.507022,2.78201,3.773541,2.382016,3.208800,2.464691,2.915016,3.727477,2.551085,2.540167,2.545788,2.162175,1.877629,1.930581,2.657053,5.281174,5.586432,4.464637,3.876054,4.229518,6.216055,1.212480,2.147192,2.074788,3.076372,2.763101,2.840231,2.149635,2.188010,2.606590,2.508952,2.421439,2.440406,2.591498,2.927888,2.731723,2.140996,2.512130,2.416360,2.372970,2.375172,2.335465,2.638626,6.301950,4.201505,8.004583,7.014178,4.499682,7.398607,5.976064,5.972657,7.899969,8.240646,1.833764,3.514224,1.364256,1.719631,1.722253,7.777244,2.413020,2.473091,2.568410,1.889459,3.160550,3.470992,3.543817,2.645243,3.003682,4.682872,4.579857,2.072847,2.018576,2.09439

In [253]:
# updated pct_missing calculation
pct_missing = pd.DataFrame(wvs_w6.isna().sum())
pct_missing = pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
pct_missing['NA_pct'] = ((pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
pct_missing

,variable,NA_count,NA_pct
0,v2,0,0.00
1,country,0,0.00
2,v4,316,0.37
3,v5,502,0.58
4,v6,958,1.12
5,v7,1502,1.75
6,v8,1433,1.67
7,v9,1255,1.46
8,v10,723,0.84
9,v11,302,0.35


### Drop variables:
- v262 (survey year)
- v230 (employment sector; difficult to meaningfully impute missings)

### Distinguish categorical from numerical variables:
- categorical:
    - v10 (target variable)
    - v12-v22 (important child qualities) 
    - v24 (trust vs careful)
    - v25-27, v30, v32-v34 (org membership)
    - v44 (desired neighbors)
    - v57 (marital status)
    - v60-65 (values/attitudes - personal and national priorities)
    - v66 (willingness to fight for country)
    - v80 (most serious problem in world)
    - v82-v83 (environmental causes)
    - v144g (religious denomination)
    - v147 religious denomination
    - v150-v151 (meaning of religion)
    - v176-v177 (security actions)
    - v179-v180 (victim of crime)
    - v187 (necessity of war)
    - v229 (employment status)
    - v237 (family savings)
    - v238 (social class)
    - v240 (sex)
    - v248 (highest educational level)
    
- identifiers:  
    - v2
    - country
    
- weights:  
    - v258
    - s018
    - s019
  
- all others are numerical
    - **note on v58 (number of children)**: impute missings for all places except Hong Kong
    - **note on v242 (age variable)**: use continuous (v242) for most analyses; use v242g for descriptive analyses only
  
### Deal with missings:
- ~mean or median imputation for numerical variables~
- ~mode imputation for categorical variables~
  
### Issues along the way:
- groupby imputation doesn't work in situations where values are 100% missing for a given country (which makes sense)
    - in situations like these, either drop the variable if it is not important, or drop the country (reducing the sample size slightly)
    - ~Drop 'ARG'~
    - ~Drop `v230`~

In [254]:
# drop Argentina; 100% missing for employment status
cases_to_drop = np.where((wvs_w6['country'] == 'ARG'))
drop_list = wvs_w6.loc[cases_to_drop].index.tolist()
len(drop_list)

1030

In [255]:
print("Current wvs_w6 shape:", wvs_w6.shape)
wvs_w6 = wvs_w6.drop(labels=drop_list)
print("New wvs_w6 shape after dropping cases from Argentina:", wvs_w6.shape)

Current wvs_w6 shape: (85898, 165)
New wvs_w6 shape after dropping cases from Argentina: (84868, 165)


In [256]:
all_vars = wvs_w6.columns.tolist()
all_vars

['v2',
 'country',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v10',
 'v11',
 'v12',
 'v13',
 'v14',
 'v15',
 'v16',
 'v17',
 'v18',
 'v19',
 'v20',
 'v21',
 'v22',
 'v24',
 'v25',
 'v26',
 'v27',
 'v30',
 'v32',
 'v33',
 'v34',
 'v44',
 'v45',
 'v47',
 'v48',
 'v49',
 'v51',
 'v52',
 'v53',
 'v54',
 'v55',
 'v56',
 'v57',
 'v58',
 'v59',
 'v60',
 'v61',
 'v62',
 'v63',
 'v64',
 'v65',
 'v66',
 'v67',
 'v68',
 'v69',
 'v70',
 'v71',
 'v72',
 'v73',
 'v74r',
 'v75',
 'v76',
 'v77',
 'v78',
 'v79',
 'v80',
 'v82',
 'v83',
 'v84',
 'v96',
 'v97',
 'v98',
 'v99',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v110',
 'v111',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v119',
 'v120',
 'v121',
 'v122',
 'v123',
 'v124',
 'v126',
 'v131',
 'v132',
 'v133',
 'v134',
 'v135',
 'v136',
 'v137',
 'v138',
 'v139',
 'v140',
 'v143',
 'v144g',
 'v147',
 'v150',
 'v151',
 'v152',
 'v153',
 'v154',
 'v155',
 'v170',
 'v171',
 'v173',
 'v174',
 'v176',
 

In [257]:
# create a list of categorical varibles
# keep 'country' for now
cat_vars = ['country', 'v10', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v24', 'v25', 
            'v26', 'v27', 'v30', 'v32', 'v33', 'v34', 'v44', 'v57', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66', 'v80',
            'v82', 'v83', 'v144g', 'v147', 'v150', 'v151', 'v176', 'v177', 'v179', 'v180', 'v187', 'v229', 'v237', 'v238',
            'v240', 'v248']
len(cat_vars)

47

In [258]:
# add all others to num_var list

num_vars = []

for variable in all_vars:
    if variable not in cat_vars:
        
        num_vars.append(variable)

print(num_vars)

['v2', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v11', 'v45', 'v47', 'v48', 'v49', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v58', 'v59', 'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74r', 'v75', 'v76', 'v77', 'v78', 'v79', 'v84', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 'v109', 'v110', 'v111', 'v113', 'v114', 'v115', 'v116', 'v117', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v126', 'v131', 'v132', 'v133', 'v134', 'v135', 'v136', 'v137', 'v138', 'v139', 'v140', 'v143', 'v152', 'v153', 'v154', 'v155', 'v170', 'v171', 'v173', 'v174', 'v181', 'v182', 'v183', 'v184', 'v188', 'v189', 'v190', 'v191', 'v192', 'v193', 'v194', 'v195', 'v196', 'v197', 'v198', 'v199', 'v200', 'v202', 'v203', 'v204', 'v205', 'v207', 'v208', 'v209', 'v210', 'v211', 'v213', 'v214', 'v216', 'v225', 'v230', 'v239', 'v242', 'v258', 's018', 's019', 'v262']


In [259]:
# put identifiers in their own list
id_vars = all_vars[0:2]
id_vars

['v2', 'country']

In [260]:
weight_vars = ['v258', 's018', 's019']
weight_vars

['v258', 's018', 's019']

In [261]:
# drop weights; keep identifiers for now
# num_vars = [var for var in num_vars if var not in id_vars]
num_vars = [var for var in num_vars if var not in weight_vars]
num_vars

['v2',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v11',
 'v45',
 'v47',
 'v48',
 'v49',
 'v51',
 'v52',
 'v53',
 'v54',
 'v55',
 'v56',
 'v58',
 'v59',
 'v67',
 'v68',
 'v69',
 'v70',
 'v71',
 'v72',
 'v73',
 'v74r',
 'v75',
 'v76',
 'v77',
 'v78',
 'v79',
 'v84',
 'v96',
 'v97',
 'v98',
 'v99',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v110',
 'v111',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v119',
 'v120',
 'v121',
 'v122',
 'v123',
 'v124',
 'v126',
 'v131',
 'v132',
 'v133',
 'v134',
 'v135',
 'v136',
 'v137',
 'v138',
 'v139',
 'v140',
 'v143',
 'v152',
 'v153',
 'v154',
 'v155',
 'v170',
 'v171',
 'v173',
 'v174',
 'v181',
 'v182',
 'v183',
 'v184',
 'v188',
 'v189',
 'v190',
 'v191',
 'v192',
 'v193',
 'v194',
 'v195',
 'v196',
 'v197',
 'v198',
 'v199',
 'v200',
 'v202',
 'v203',
 'v204',
 'v205',
 'v207',
 'v208',
 'v209',
 'v210',
 'v211',
 'v213',
 'v214',
 'v216',
 'v225',
 'v230',
 'v239',
 'v242',
 'v262']

In [262]:
# replace v2 with country in num_vars list
del num_vars[0]
num_vars.insert(0, 'country')
num_vars

['country',
 'v4',
 'v5',
 'v6',
 'v7',
 'v8',
 'v9',
 'v11',
 'v45',
 'v47',
 'v48',
 'v49',
 'v51',
 'v52',
 'v53',
 'v54',
 'v55',
 'v56',
 'v58',
 'v59',
 'v67',
 'v68',
 'v69',
 'v70',
 'v71',
 'v72',
 'v73',
 'v74r',
 'v75',
 'v76',
 'v77',
 'v78',
 'v79',
 'v84',
 'v96',
 'v97',
 'v98',
 'v99',
 'v100',
 'v101',
 'v102',
 'v103',
 'v104',
 'v105',
 'v106',
 'v107',
 'v108',
 'v109',
 'v110',
 'v111',
 'v113',
 'v114',
 'v115',
 'v116',
 'v117',
 'v119',
 'v120',
 'v121',
 'v122',
 'v123',
 'v124',
 'v126',
 'v131',
 'v132',
 'v133',
 'v134',
 'v135',
 'v136',
 'v137',
 'v138',
 'v139',
 'v140',
 'v143',
 'v152',
 'v153',
 'v154',
 'v155',
 'v170',
 'v171',
 'v173',
 'v174',
 'v181',
 'v182',
 'v183',
 'v184',
 'v188',
 'v189',
 'v190',
 'v191',
 'v192',
 'v193',
 'v194',
 'v195',
 'v196',
 'v197',
 'v198',
 'v199',
 'v200',
 'v202',
 'v203',
 'v204',
 'v205',
 'v207',
 'v208',
 'v209',
 'v210',
 'v211',
 'v213',
 'v214',
 'v216',
 'v225',
 'v230',
 'v239',
 'v242',
 'v262']

In [263]:
len(num_vars)

115

In [264]:
# drop v242g
#del num_vars[114]
#num_vars

### Deal with categorical variable missings (mode imputation)

In [265]:
categorical_df = wvs_w6[cat_vars]
categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84868 entries, 0 to 85897
Data columns (total 47 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  84868 non-null  object 
 1   v10      84156 non-null  float64
 2   v12      84854 non-null  float64
 3   v13      84862 non-null  float64
 4   v14      84861 non-null  float64
 5   v15      84851 non-null  float64
 6   v16      84859 non-null  float64
 7   v17      84855 non-null  float64
 8   v18      84856 non-null  float64
 9   v19      84850 non-null  float64
 10  v20      84853 non-null  float64
 11  v21      84849 non-null  float64
 12  v22      84865 non-null  float64
 13  v24      82599 non-null  float64
 14  v25      84268 non-null  float64
 15  v26      84203 non-null  float64
 16  v27      84153 non-null  float64
 17  v30      84061 non-null  float64
 18  v32      84079 non-null  float64
 19  v33      83979 non-null  float64
 20  v34      84029 non-null  float64
 21  v44      848

In [266]:
cat_vars_summary1 = categorical_df.describe()
cat_vars_summary1

,v10,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v57,v60,v61,v62,v63,v64,v65,v66,v80,v82,v83,v144g,v147,v150,v151,v176,v177,v179,v180,v187,v229,v237,v238,v240,v248
count,84156.000000,84854.000000,84862.000000,84861.000000,84851.000000,84859.000000,84855.000000,84856.000000,84850.000000,84853.000000,84849.000000,84865.000000,82599.000000,84268.000000,84203.000000,84153.000000,84061.000000,84079.000000,83979.000000,84029.000000,84848.000000,84663.000000,83189.000000,80947.000000,83496.000000,81653.000000,83601.00000,81922.000000,76417.000000,83843.000000,83916.000000,83637.000000,83626.000000,82004.000000,79652.000000,77802.000000,82939.000000,83179.000000,83531.000000,82263.000000,79459.000000,84419.000000,79001.000000,82472.000000,84828.000000,84143.000000
mean,1.837136,1.488604,1.388808,1.280824,1.770150,1.316572,1.598657,1.598508,1.597596,1.660589,1.582883,1.729040,1.756014,0.522773,0.328242,0.238150,0.131381,0.189286,0.111576,0.165621,1.829967,2.729421,1.706872,2.562825,2.049571,2.495120,1.87749,2.737336,1.328147,2.163341,1.877377,1.930748,3.537620,1.363933,1.718249,1.721434,2.649067,3.000793,4.687350,4.587032,1.654275,3.412135,2.082505,3.316944,1.521432,5.642466
std,0.722141,0.499873,0.487482,0.449404,0.420739,0.465142,0.490173,0.490203,0.490385,0.473512,0.493086,0.444458,0.429487,0.786313,0.663646,0.574333,0.421341,0.512278,0.384240,0.487039,0.375664,2.193218,0.995356,1.035656,1.034930,1.045617,1.16513,1.158795,0.469542,1.513030,0.328005,0.253883,2.989893,0.585201,0.561348,0.564199,1.968983,2.000012,1.073716,1.217106,0.475607,2.156241,0.931096,0.999451,0.499543,2.412580
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.00000,2.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,1.000000,1.000000,1.000000,3.000000,1.000000,4.000000
50%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,3.000000,2.000000,3.000000,1.00000,3.000000,1.000000,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,1.000000,5.000000,5.000000,5.000000,2.000000,3.000000,2.000000,3.000000,2.000000,6.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.000000,2.000000,3.000000,3.000000,3.000000,3.00000,4.000000,2.000000,3.000000,2.000000,2.000000,5.000000,2.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000,2.000000,5.000000,3.000000,4.000000,2.000000,7.000000
max,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,6.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,2.000000,5.000000,2.000000,2.000000,9.000000,3.000000,4.000000,4.000000,5.000000,5.000000,5.000000,5.000000,2.000000,8.000000,4.000000,5.000000,2.000000,9.000000


In [267]:
# check out current proportion of missings
cat_pct_missing = pd.DataFrame(categorical_df.isna().sum())
cat_pct_missing = cat_pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
cat_pct_missing['NA_pct'] = ((cat_pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
cat_pct_missing

,variable,NA_count,NA_pct
0,country,0,0.00
1,v10,712,0.84
2,v12,14,0.02
3,v13,6,0.01
4,v14,7,0.01
5,v15,17,0.02
6,v16,9,0.01
7,v17,13,0.02
8,v18,12,0.01
9,v19,18,0.02


In [268]:
# deal with missings for one categorical variable (v10); impute mode
#wvs_w6['v10'] = wvs_w6['v10'].fillna(wvs_w6.groupby('country')['v10'].transform(lambda x: pd.Series.mode(x)[0]))

In [269]:
# impute group mode for categorical variables' missing values

for var in cat_vars: 
    
    if var == 'country': continue
    
    wvs_w6[var] = wvs_w6[var].fillna(wvs_w6.groupby('country')[var].transform(lambda x: pd.Series.mode(x)[0]))

In [270]:
# check out updated proportion of missings
categorical_df = wvs_w6[cat_vars]

cat_pct_missing = pd.DataFrame(categorical_df.isna().sum())
cat_pct_missing = cat_pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
cat_pct_missing['NA_pct'] = ((cat_pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
cat_pct_missing

,variable,NA_count,NA_pct
0,country,0,0.0
1,v10,0,0.0
2,v12,0,0.0
3,v13,0,0.0
4,v14,0,0.0
5,v15,0,0.0
6,v16,0,0.0
7,v17,0,0.0
8,v18,0,0.0
9,v19,0,0.0


In [271]:
# new quick summary stats aren't too different from their previous values, even for variables where more than 5% of values
# were missing (v66, v150, v151, v187, v237)
cat_vars_summary2 = wvs_w6[cat_vars].describe()
cat_vars_summary2

,v10,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v57,v60,v61,v62,v63,v64,v65,v66,v80,v82,v83,v144g,v147,v150,v151,v176,v177,v179,v180,v187,v229,v237,v238,v240,v248
count,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000
mean,1.838243,1.488665,1.388804,1.280813,1.770196,1.316550,1.598600,1.598541,1.597681,1.660638,1.582964,1.729050,1.760357,0.519737,0.325670,0.236143,0.130131,0.187527,0.110407,0.163984,1.830007,2.725774,1.692888,2.579630,2.045530,2.507906,1.866722,2.762502,1.319885,2.156360,1.878753,1.931753,3.544717,1.360949,1.728119,1.735483,2.632229,2.978649,4.692275,4.599708,1.651070,3.404840,2.058762,3.321594,1.521221,5.662264
std,0.719381,0.499874,0.487481,0.449399,0.420709,0.465133,0.490184,0.490196,0.490368,0.473496,0.493072,0.444453,0.426868,0.785156,0.661674,0.572323,0.419527,0.510215,0.382391,0.484903,0.375629,2.192399,0.990365,1.026226,1.034627,1.037706,1.162559,1.159421,0.466435,1.513188,0.326416,0.252171,3.006576,0.581388,0.551159,0.549728,1.965907,1.999898,1.065936,1.200397,0.476635,2.156039,0.910450,0.993360,0.499552,2.417918
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,1.000000,1.000000,1.000000,3.000000,1.000000,4.000000
50%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,3.000000,2.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,1.000000,1.000000,5.000000,5.000000,2.000000,3.000000,2.000000,3.000000,2.000000,6.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.000000,2.000000,3.000000,3.000000,3.000000,3.000000,4.000000,2.000000,3.000000,2.000000,2.000000,5.000000,2.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000,2.000000,5.000000,2.000000,4.000000,2.000000,8.000000
max,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,6.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,2.000000,5.000000,2.000000,2.000000,9.000000,3.000000,4.000000,4.000000,5.000000,5.000000,5.000000,5.000000,2.000000,8.000000,4.000000,5.000000,2.000000,9.000000


In [272]:
# repeat of previous variable info for easier comparison
cat_vars_summary1

,v10,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v57,v60,v61,v62,v63,v64,v65,v66,v80,v82,v83,v144g,v147,v150,v151,v176,v177,v179,v180,v187,v229,v237,v238,v240,v248
count,84156.000000,84854.000000,84862.000000,84861.000000,84851.000000,84859.000000,84855.000000,84856.000000,84850.000000,84853.000000,84849.000000,84865.000000,82599.000000,84268.000000,84203.000000,84153.000000,84061.000000,84079.000000,83979.000000,84029.000000,84848.000000,84663.000000,83189.000000,80947.000000,83496.000000,81653.000000,83601.00000,81922.000000,76417.000000,83843.000000,83916.000000,83637.000000,83626.000000,82004.000000,79652.000000,77802.000000,82939.000000,83179.000000,83531.000000,82263.000000,79459.000000,84419.000000,79001.000000,82472.000000,84828.000000,84143.000000
mean,1.837136,1.488604,1.388808,1.280824,1.770150,1.316572,1.598657,1.598508,1.597596,1.660589,1.582883,1.729040,1.756014,0.522773,0.328242,0.238150,0.131381,0.189286,0.111576,0.165621,1.829967,2.729421,1.706872,2.562825,2.049571,2.495120,1.87749,2.737336,1.328147,2.163341,1.877377,1.930748,3.537620,1.363933,1.718249,1.721434,2.649067,3.000793,4.687350,4.587032,1.654275,3.412135,2.082505,3.316944,1.521432,5.642466
std,0.722141,0.499873,0.487482,0.449404,0.420739,0.465142,0.490173,0.490203,0.490385,0.473512,0.493086,0.444458,0.429487,0.786313,0.663646,0.574333,0.421341,0.512278,0.384240,0.487039,0.375664,2.193218,0.995356,1.035656,1.034930,1.045617,1.16513,1.158795,0.469542,1.513030,0.328005,0.253883,2.989893,0.585201,0.561348,0.564199,1.968983,2.000012,1.073716,1.217106,0.475607,2.156241,0.931096,0.999451,0.499543,2.412580
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.00000,2.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,1.000000,1.000000,1.000000,3.000000,1.000000,4.000000
50%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,3.000000,2.000000,3.000000,1.00000,3.000000,1.000000,1.000000,2.000000,2.000000,3.000000,1.000000,2.000000,2.000000,1.000000,5.000000,5.000000,5.000000,2.000000,3.000000,2.000000,3.000000,2.000000,6.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.000000,2.000000,3.000000,3.000000,3.000000,3.00000,4.000000,2.000000,3.000000,2.000000,2.000000,5.000000,2.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000,2.000000,5.000000,3.000000,4.000000,2.000000,7.000000
max,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,6.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,2.000000,5.000000,2.000000,2.000000,9.000000,3.000000,4.000000,4.000000,5.000000,5.000000,5.000000,5.000000,2.000000,8.000000,4.000000,5.000000,2.000000,9.000000


### Deal with numerical variable missings (median imputation)

In [273]:
numerical_df = wvs_w6[num_vars]
numerical_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84868 entries, 0 to 85897
Data columns (total 115 columns):
 #   Column   Dtype  
---  ------   -----  
 0   country  object 
 1   v4       float64
 2   v5       float64
 3   v6       float64
 4   v7       float64
 5   v8       float64
 6   v9       float64
 7   v11      float64
 8   v45      float64
 9   v47      float64
 10  v48      float64
 11  v49      float64
 12  v51      float64
 13  v52      float64
 14  v53      float64
 15  v54      float64
 16  v55      float64
 17  v56      float64
 18  v58      float64
 19  v59      float64
 20  v67      float64
 21  v68      float64
 22  v69      float64
 23  v70      float64
 24  v71      float64
 25  v72      float64
 26  v73      float64
 27  v74r     float64
 28  v75      float64
 29  v76      float64
 30  v77      float64
 31  v78      float64
 32  v79      float64
 33  v84      float64
 34  v96      float64
 35  v97      float64
 36  v98      float64
 37  v99      float64
 38  v100 

In [274]:
num_vars_summary1 = numerical_df.describe()
num_vars_summary1

,v4,v5,v6,v7,v8,v9,v11,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v58,v59,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v152,v153,v154,v155,v170,v171,v173,v174,v181,v182,v183,v184,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v230,v239,v242,v262
count,84554.000000,84368.000000,83916.000000,83380.000000,83452.000000,83631.000000,84572.000000,83309.000000,81576.000000,82059.000000,81826.000000,80969.000000,82374.000000,81142.000000,80508.000000,83501.000000,83072.000000,83235.000000,84307.000000,81748.000000,81536.000000,81272.000000,81980.000000,82352.000000,82849.000000,82462.000000,84868.000000,82125.000000,81826.000000,82621.000000,82636.000000,82838.000000,83938.000000,82126.000000,79792.000000,82750.000000,81841.000000,82731.000000,80466.000000,84282.000000,83715.000000,83808.000000,82641.000000,78948.000000,78706.000000,82104.000000,82163.000000,82257.000000,82990.000000,82905.000000,81672.000000,81844.000000,79949.000000,80596.000000,79737.000000,78688.000000,81033.000000,77773.000000,76282.000000,78331.000000,74478.000000,81219.000000,79291.000000,81685.000000,81653.000000,79277.000000,79862.000000,80884.000000,80334.000000,82145.000000,82718.000000,83274.000000,82858.000000,76456.000000,76843.000000,77096.000000,83093.000000,80810.000000,77909.000000,75516.000000,79670.000000,78216.000000,81794.000000,81813.000000,83618.000000,83178.000000,83296.000000,83319.000000,82075.000000,81798.000000,80257.000000,78548.000000,81410.000000,82239.000000,82529.000000,83231.000000,83575.000000,83255.000000,80511.000000,81929.000000,82481.000000,82396.000000,83468.000000,83493.000000,83455.000000,83442.000000,82390.000000,83379.000000,80203.000000,83698.000000,64214.000000,81892.000000,84759.000000,84868.000000
mean,1.095773,1.678148,1.882120,2.636376,1.509203,1.899164,2.096781,2.037391,2.094256,1.604748,1.654694,2.433697,2.943696,2.563617,2.217780,7.061221,5.678773,1.854112,5.865124,2.288998,1.378177,1.506977,2.784923,3.764924,2.368103,3.213662,2.463661,2.911696,3.728558,2.548226,2.538325,2.543495,2.654543,5.287546,5.579131,4.459734,3.866180,4.229672,6.220205,1.213960,2.147572,2.076902,3.078085,2.767695,2.845577,2.144500,2.178864,2.604581,2.505230,2.413509,2.431604,2.587716,2.924564,2.728398,2.141001,2.508006,2.409932,2.372160,2.372408,2.335831,2.635906,6.309275,4.206177,7.997650,7.014721,4.518486,7.398375,5.971799,5.969490,7.891265,8.235559,1.828158,7.784656,2.406024,2.464024,2.564426,1.883396,3.168284,3.471640,3.543924,2.066449,2.012133,2.084028,2.069182,3.435050,3.424547,3.340124,3.067716,7.614280,7.745397,5.504155,5.373886,4.725709,7.246647,2.779059,2.656294,1.817039,1.960687,3.220044,3.214149,4.500030,2.244041,2.066720,2.950343,1.966030,1.560988,1.793895,1.505355,2.075496,2.005819,1.974227,4.805976,41.934744,2012.067528
std,0.349437,0.735349,0.834674,0.983415,0.790092,1.054379,0.849571,0.898298,0.845911,0.782249,0.750227,0.993372,0.932298,0.982206,0.932549,2.252129,2.646825,1.787741,2.479937,0.851346,0.605528,0.695974,1.414083,1.549553,1.299312,1.537746,1.222339,1.459650,1.602023,1.358090,1.303581,1.431339,0.974225,2.968557,2.786708,2.916476,2.626167,2.891731,2.787490,0.515244,0.828792,0.804038,0.792367,0.860665,0.860125,1.027766,0.937427,0.874314,0.878908,0.942141,0.943572,0.956856,0.886643,0.936674,0.858562,0.876540,0.937851,0.872764,0.893854,0.893739,0.955513,2.972897,2.946433,2.480662,2.750519,3.117986,2.579437,3.022798,3.006721,2.584049,2.122405,0.855746,2.948186,1.061011,1.078570,1.011087,0.793674,0.902727,0.780422,0.765779,1.134450,1.099112,1.101433,1.098498,0.875468,0.871438,0.922945,1.043187,2.274787,2.226888,2.812561,2.738946,2.885297,2.364525,2.557504,2.432046,1.827227,1.943840,3.009255,2.759405,3.083391,2.214168,2.049332,2.607650,1.908117

In [275]:
# check out current proportion of missings for num_vars
num_pct_missing = pd.DataFrame(numerical_df.isna().sum())
num_pct_missing = num_pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
num_pct_missing['NA_pct'] = ((num_pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
num_pct_missing

,variable,NA_count,NA_pct
0,country,0,0.00
1,v4,314,0.37
2,v5,500,0.59
3,v6,952,1.12
4,v7,1488,1.75
5,v8,1416,1.67
6,v9,1237,1.46
7,v11,296,0.35
8,v45,1559,1.84
9,v47,3292,3.88


In [276]:
# impute group median for numerical variables' missing values

for var in num_vars: 
    
    if var == 'country': continue # v2 is the country code grouping variable in this variable list
    
    wvs_w6[var] = wvs_w6[var].fillna(wvs_w6.groupby('country')[var].transform('median'))

In [277]:
# updated proportion of missings
numerical_df = wvs_w6[num_vars]

num_pct_missing = pd.DataFrame(numerical_df.isna().sum())
num_pct_missing = num_pct_missing.reset_index().rename(columns = {'index':'variable', 0:'NA_count'})
num_pct_missing['NA_pct'] = ((num_pct_missing['NA_count'] / len(wvs_w6)) * 100).round(decimals=2)
num_pct_missing

,variable,NA_count,NA_pct
0,country,0,0.00
1,v4,0,0.00
2,v5,0,0.00
3,v6,0,0.00
4,v7,0,0.00
5,v8,0,0.00
6,v9,0,0.00
7,v11,0,0.00
8,v45,0,0.00
9,v47,0,0.00


In [278]:
# new quick summary stats aren't too different from their previous values, even for variables where more than 5% of values
# were missing (v66, v150, v151, v187, v237)
num_vars_summary2 = wvs_w6[num_vars].describe()
num_vars_summary2

,v4,v5,v6,v7,v8,v9,v11,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v58,v59,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v152,v153,v154,v155,v170,v171,v173,v174,v181,v182,v183,v184,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v230,v239,v242,v262
count,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,83868.000000,84868.000000,84868.000000,84868.000000,84868.00000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,83808.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.00000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,84868.000000,83368.000000,84868.000000,84868.000000,84868.000000
mean,1.095419,1.678147,1.883089,2.638957,1.507612,1.911180,2.097104,2.034230,2.096031,1.591012,1.659483,2.438104,2.945350,2.566315,2.215829,7.057642,5.681175,1.853317,5.865992,2.298357,1.364660,1.50740,2.786186,3.759768,2.366546,3.210939,2.463661,2.913595,3.727683,2.548793,2.539885,2.545883,2.655005,5.270043,5.587854,4.440849,3.828463,4.208053,6.227836,1.213060,2.146486,2.076825,3.077297,2.772706,2.847764,2.157904,2.174459,2.605717,2.506292,2.410378,2.428925,2.591978,2.930221,2.733468,2.137001,2.511559,2.408800,2.378069,2.373062,2.331421,2.627551,6.333388,4.146357,8.031861,7.046083,4.480582,7.456745,5.984588,5.942416,7.924047,8.257305,1.827308,7.735743,2.435806,2.495852,2.587442,1.884856,3.181529,3.504006,3.59146,2.073526,2.023319,2.083547,2.075517,3.441321,3.434227,3.349460,3.074586,7.636494,7.760086,5.485472,5.364908,4.687014,7.256392,2.756033,2.640359,1.807584,1.946647,3.196270,3.211576,4.507164,2.225291,2.053860,2.946234,1.954141,1.562827,1.799807,1.507659,2.089492,2.000318,1.990332,4.776111,41.935370,2012.067528
std,0.348838,0.734056,0.830235,0.977773,0.786147,1.058052,0.848421,0.895822,0.833969,0.776054,0.741124,0.979456,0.919225,0.966108,0.912801,2.236561,2.622765,1.781672,2.472574,0.844910,0.598352,0.69117,1.394015,1.532720,1.287698,1.520376,1.222339,1.441689,1.580177,1.345265,1.289629,1.420624,0.970562,2.939981,2.710778,2.889120,2.601119,2.872877,2.731151,0.514091,0.823716,0.799515,0.783165,0.841691,0.834730,1.019626,0.927866,0.864883,0.872501,0.934612,0.931727,0.944293,0.879063,0.924954,0.836077,0.854695,0.925985,0.847966,0.864018,0.867730,0.930296,2.923434,2.897609,2.446958,2.712849,3.049510,2.524963,2.973487,2.964890,2.555416,2.103954,0.849574,2.960041,1.035078,1.047403,0.977169,0.786062,0.890121,0.761800,0.73668,1.110534,1.072841,1.089467,1.087676,0.872333,0.866322,0.919105,1.039092,2.244122,2.191276,2.748274,2.640418,2.840485,2.332326,2.532215,2.415814,1.816917,1.931664,2.985509,2.739250,3.059846,2.193344,2.037932,2.594476,1.897361,0.781

In [279]:
# previous stats for comparison
num_vars_summary1

,v4,v5,v6,v7,v8,v9,v11,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v58,v59,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v152,v153,v154,v155,v170,v171,v173,v174,v181,v182,v183,v184,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v230,v239,v242,v262
count,84554.000000,84368.000000,83916.000000,83380.000000,83452.000000,83631.000000,84572.000000,83309.000000,81576.000000,82059.000000,81826.000000,80969.000000,82374.000000,81142.000000,80508.000000,83501.000000,83072.000000,83235.000000,84307.000000,81748.000000,81536.000000,81272.000000,81980.000000,82352.000000,82849.000000,82462.000000,84868.000000,82125.000000,81826.000000,82621.000000,82636.000000,82838.000000,83938.000000,82126.000000,79792.000000,82750.000000,81841.000000,82731.000000,80466.000000,84282.000000,83715.000000,83808.000000,82641.000000,78948.000000,78706.000000,82104.000000,82163.000000,82257.000000,82990.000000,82905.000000,81672.000000,81844.000000,79949.000000,80596.000000,79737.000000,78688.000000,81033.000000,77773.000000,76282.000000,78331.000000,74478.000000,81219.000000,79291.000000,81685.000000,81653.000000,79277.000000,79862.000000,80884.000000,80334.000000,82145.000000,82718.000000,83274.000000,82858.000000,76456.000000,76843.000000,77096.000000,83093.000000,80810.000000,77909.000000,75516.000000,79670.000000,78216.000000,81794.000000,81813.000000,83618.000000,83178.000000,83296.000000,83319.000000,82075.000000,81798.000000,80257.000000,78548.000000,81410.000000,82239.000000,82529.000000,83231.000000,83575.000000,83255.000000,80511.000000,81929.000000,82481.000000,82396.000000,83468.000000,83493.000000,83455.000000,83442.000000,82390.000000,83379.000000,80203.000000,83698.000000,64214.000000,81892.000000,84759.000000,84868.000000
mean,1.095773,1.678148,1.882120,2.636376,1.509203,1.899164,2.096781,2.037391,2.094256,1.604748,1.654694,2.433697,2.943696,2.563617,2.217780,7.061221,5.678773,1.854112,5.865124,2.288998,1.378177,1.506977,2.784923,3.764924,2.368103,3.213662,2.463661,2.911696,3.728558,2.548226,2.538325,2.543495,2.654543,5.287546,5.579131,4.459734,3.866180,4.229672,6.220205,1.213960,2.147572,2.076902,3.078085,2.767695,2.845577,2.144500,2.178864,2.604581,2.505230,2.413509,2.431604,2.587716,2.924564,2.728398,2.141001,2.508006,2.409932,2.372160,2.372408,2.335831,2.635906,6.309275,4.206177,7.997650,7.014721,4.518486,7.398375,5.971799,5.969490,7.891265,8.235559,1.828158,7.784656,2.406024,2.464024,2.564426,1.883396,3.168284,3.471640,3.543924,2.066449,2.012133,2.084028,2.069182,3.435050,3.424547,3.340124,3.067716,7.614280,7.745397,5.504155,5.373886,4.725709,7.246647,2.779059,2.656294,1.817039,1.960687,3.220044,3.214149,4.500030,2.244041,2.066720,2.950343,1.966030,1.560988,1.793895,1.505355,2.075496,2.005819,1.974227,4.805976,41.934744,2012.067528
std,0.349437,0.735349,0.834674,0.983415,0.790092,1.054379,0.849571,0.898298,0.845911,0.782249,0.750227,0.993372,0.932298,0.982206,0.932549,2.252129,2.646825,1.787741,2.479937,0.851346,0.605528,0.695974,1.414083,1.549553,1.299312,1.537746,1.222339,1.459650,1.602023,1.358090,1.303581,1.431339,0.974225,2.968557,2.786708,2.916476,2.626167,2.891731,2.787490,0.515244,0.828792,0.804038,0.792367,0.860665,0.860125,1.027766,0.937427,0.874314,0.878908,0.942141,0.943572,0.956856,0.886643,0.936674,0.858562,0.876540,0.937851,0.872764,0.893854,0.893739,0.955513,2.972897,2.946433,2.480662,2.750519,3.117986,2.579437,3.022798,3.006721,2.584049,2.122405,0.855746,2.948186,1.061011,1.078570,1.011087,0.793674,0.902727,0.780422,0.765779,1.134450,1.099112,1.101433,1.098498,0.875468,0.871438,0.922945,1.043187,2.274787,2.226888,2.812561,2.738946,2.885297,2.364525,2.557504,2.432046,1.827227,1.943840,3.009255,2.759405,3.083391,2.214168,2.049332,2.607650,1.908117

**Missings are still present for:**
v58 (100% missing for HKG) - number of children item
v116 (100% missing for QAT) - confidence in political parties
v230 (100% missing for UZB) - employment sector item

**Solutions to deal with missing values:**
- keep v58 and v116 _and_ keep HKG and QAT
    - include HKG and QAT in clusering and UMAP projection
    - exclude HKG and QAT from logistic regression and random forest modeling
- drop v230
    - NTH but not critical
    - this way UZB remains in the sample

In [280]:
wvs_w6.drop(['v230'], axis = 1, inplace = True)

In [281]:
wvs_w6.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84868 entries, 0 to 85897
Data columns (total 164 columns):
 #   Column   Dtype  
---  ------   -----  
 0   v2       int64  
 1   country  object 
 2   v4       float64
 3   v5       float64
 4   v6       float64
 5   v7       float64
 6   v8       float64
 7   v9       float64
 8   v10      float64
 9   v11      float64
 10  v12      float64
 11  v13      float64
 12  v14      float64
 13  v15      float64
 14  v16      float64
 15  v17      float64
 16  v18      float64
 17  v19      float64
 18  v20      float64
 19  v21      float64
 20  v22      float64
 21  v24      float64
 22  v25      float64
 23  v26      float64
 24  v27      float64
 25  v30      float64
 26  v32      float64
 27  v33      float64
 28  v34      float64
 29  v44      float64
 30  v45      float64
 31  v47      float64
 32  v48      float64
 33  v49      float64
 34  v51      float64
 35  v52      float64
 36  v53      float64
 37  v54      float64
 38  v55  

### Engineer age groups based on responses to `v242`

- recode age variable `V242` into groups 
    - 0-14 years (children) = _[excluded from analysis]_
    - 15-24 years (early working age) = _[group 1]_
    - 25-54 years (prime working age) = _[group 2]_
    - 55-64 years (mature working age) = _[group 3]_
    - 65 years and over (elderly) = _[group 4]_

In [282]:
# recode age variable into groups
bins = [14, 24, 54, 64, 120]
labels = [1, 2, 3, 4]
wvs_w6['v242g'] = pd.cut(wvs_w6['v242'], bins=bins, labels=labels)
wvs_w6['v242g'].value_counts().sort_index()

1    14415
2    49823
3    10961
4     9669
Name: v242g, dtype: int64

In [283]:
# get index for v242
wvs_w6.columns.get_loc('v242')

158

In [284]:
# remove and re-insert v242g into the dataframe
col_name = 'v242g'
v242g = wvs_w6.pop(col_name)
wvs_w6.insert(159, col_name, v242g)
wvs_w6.head()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v237,v238,v239,v240,v242,v242g,v248,v258,s018,s019,v262
0,12,ALG,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,4.0,6.0,0.0,10.0,2.0,3.0,1.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,3.0,2.0,2.0,2.0,4.0,8.0,7.0,6.0,8.0,7.0,5.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,3.0,8.0,5.0,6.0,9.0,3.0,4.0,7.0,6.0,7.0,2.0,5.0,1.0,1.0,1.0,10.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,7.0,8.0,3.0,5.0,6.0,9.0,6.0,6.0,1.0,1.0,1.0,1.0,3.0,1.0,6.0,5.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,1.0,4.0,5.0,1.0,21.0,1,7.0,1.0,0.833333,1.25,2014
1,12,ALG,1.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,6.0,8.0,6.0,0.0,10.0,2.0,1.0,2.0,3.0,4.0,3.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,3.0,7.0,5.0,5.0,4.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,3.0,1.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,8.0,8.0,8.0,9.0,2.0,6.0,4.0,2.0,4.0,1.0,5.0,1.0,2.0,1.0,10.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,8.0,4.0,6.0,4.0,8.0,3.0,4.0,7.0,1.0,1.0,1.0,1.0,1.0,3.0,5.0,1.0,2.0,2.0,2.0,2.0,3.0,6.0,2.0,3.0,6.0,2.0,24.0,1,7.0,1.0,0.833333,1.25,2014
2,12,ALG,1.0,3.0,2.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,6.0,8.0,6.0,0.0,6.0,2.0,4.0,1.0,2.0,1.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,7.0,7.0,7.0,5.0,7.0,5.0,1.0,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,2.0,2.0,4.0,3.0,2.0,3.0,2.0,4.0,2.0,2.0,7.0,4.0,8.0,3.0,3.0,6.0,9.0,5.0,6.0,1.0,5.0,1.0,2.0,1.0,6.0,2.0,3.0,1.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,7.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,3.0,2.0,4.0,2.0,3.0,1.0,4.0,6.0,2.0,26.0,2,5.0,1.0,0.833333,1.25,2014
3,12,ALG,1.0,1.0,3.0,4.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,6.0,8.0,6.0,0.0,6.0,2.0,1.0,3.0,1.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,3.0,1.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,9.0,5.0,6.0,4.0,6.0,8.0,1.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,4.0,2.0,4.0,2.0,3.0,3.0,4.0,2.0,2.0,3.0,1.0,2.0,4.0,3.0,2.0,7.0,9.0,5.0,5.0,7.0,3.0,8.0,7.0,8.0,2.0,5.0,1.0,2.0,1.0,10.0,2.0,3.0,4.0,1.0,2.0,2.0,2.0,5.0,5.0,1.0,5.0,2.0,3.0,3.0,3.0,2.0,2.0,3.0,3.0,3.0,6.0,6.0,3.0,5.0,5.0,7.0,4.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,4.0,4.0,5.0,2.0,28.0,2,6.0,1.0,0.833333,1.25,2014
4,12,ALG,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,6.0,6.0,1.0,3.0,4.0,2.0,1.0,2.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,2.0,5.0,1.0,2.0,3.0,4.0,3.0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,8.0,4.0,7.0,4.0,6.0,6.0,2.0,2.0,3.0,4.0,2.0,3.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,4.0,3.0,2.0,3.0,2.0,8.0,4.0,7.0,3.0,3.0,8.0,6.0,5.0,6.0,2.0,5.0,1.0,1.0,1.0,10.0,2.0,3.0,2.0,2.0,2.0,3.0,3.0,5.0,5.0,5.0,5.0,2.0,3.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,6.0,2.0,4.0,4.0,6.0,6.0,6.0,5.0,7.0,1.0,1.0,1.

In [285]:
# check to see that ages are in the correct group
# anyone under 25-yrs-old (should be group 1)
wvs_w6[wvs_w6['v242'] < 25.0].sample()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v237,v238,v239,v240,v242,v242g,v248,v258,s018,s019,v262
84991,716,ZIM,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,6.0,6.0,6.0,0.0,5.0,1.0,4.0,3.0,4.0,4.0,2.0,1.0,3.0,3.0,1.0,3.0,4.0,2.0,4.0,2.0,2.0,5.0,1.0,4.0,6.0,1.0,2.0,2.0,4.0,8.0,8.0,1.0,2.0,1.0,9.0,2.0,2.0,4.0,4.0,4.0,3.0,1.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,5.0,7.0,1.0,1.0,8.0,1.0,8.0,1.0,7.0,2.0,2.0,1.0,1.0,1.0,10.0,1.0,2.0,3.0,3.0,3.0,4.0,4.0,5.0,5.0,5.0,5.0,1.0,1.0,4.0,2.0,2.0,2.0,3.0,4.0,2.0,6.0,6.0,5.0,5.0,6.0,5.0,3.0,4.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,6.0,1.0,3.0,1.0,1.0,1.0,1.0,6.0,2.0,3.0,5.0,1.0,22.0,1,5.0,0.4329,0.2886,0.4329,2012


In [286]:
# 25-yr-old (should be group 2)
wvs_w6[wvs_w6['v242'] == 25.0].sample()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v237,v238,v239,v240,v242,v242g,v248,v258,s018,s019,v262
11039,196,CYP,1.0,1.0,3.0,4.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,1.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,2.0,10.0,1.0,2.0,2.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,5.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,1.0,10.0,1.0,1.0,1.0,10.0,1.0,4.0,4.0,4.0,3.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,10.0,1.0,5.0,1.0,1.0,1.0,10.0,3.0,3.0,1.0,2.0,4.0,4.0,4.0,1.0,1.0,5.0,5.0,1.0,1.0,1.0,3.0,2.0,3.0,3.0,3.0,1.0,10.0,10.0,1.0,4.0,1.0,10.0,1.0,1.0,1.0,3.0,6.0,1.0,8.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,3.0,5.0,2.0,25.0,2,7.0,0.44156,0.44156,0.66234,2011


In [287]:
# 55-yr-old (should be group 3)
wvs_w6[wvs_w6['v242'] == 55.0].sample()

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v237,v238,v239,v240,v242,v242g,v248,v258,s018,s019,v262
31788,392,JPN,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,5.0,6.0,1.0,2.0,10.0,3.0,1.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,2.0,5.0,5.0,4.0,5.0,6.0,5.0,3.0,1.0,2.0,1.0,1.0,2.0,2.0,6.0,1.0,3.0,3.0,6.0,5.0,1.0,1.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,1.0,3.0,3.0,3.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,6.0,1.0,9.0,5.0,1.0,8.0,1.0,2.0,9.0,10.0,1.0,0.0,2.0,2.0,2.0,5.0,3.0,3.0,3.0,2.0,3.0,4.0,4.0,1.0,1.0,5.0,5.0,1.0,1.0,1.0,1.0,2.0,4.0,4.0,4.0,4.0,9.0,10.0,1.0,1.0,1.0,4.0,1.0,1.0,1.0,1.0,10.0,5.0,5.0,3.0,2.0,1.0,1.0,1.0,2.0,2.0,3.0,2.0,5.0,2.0,3.0,3.0,2.0,55.0,3,7.0,1.0,0.409333,0.613999,2010


In [288]:
# anyone 65-yrs-old and up (should be group 4)
wvs_w6[wvs_w6['v242'] >= 65.0].sample(5)

,v2,country,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v24,v25,v26,v27,v30,v32,v33,v34,v44,v45,v47,v48,v49,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74r,v75,v76,v77,v78,v79,v80,v82,v83,v84,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v113,v114,v115,v116,v117,v119,v120,v121,v122,v123,v124,v126,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v143,v144g,v147,v150,v151,v152,v153,v154,v155,v170,v171,v173,v174,v176,v177,v179,v180,v181,v182,v183,v184,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v202,v203,v204,v205,v207,v208,v209,v210,v211,v213,v214,v216,v225,v229,v237,v238,v239,v240,v242,v242g,v248,v258,s018,s019,v262
4237,36,AUL,1.0,2.0,2.0,2.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,10.0,10.0,1.0,2.0,6.0,2.0,3.0,2.0,3.0,1.0,4.0,1.0,3.0,2.0,1.0,3.0,5.0,2.0,4.0,3.0,4.0,4.0,1.0,2.0,3.0,1.0,2.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0,10.0,1.0,3.0,2.0,3.0,2.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,4.0,3.0,3.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,9.0,1.0,10.0,10.0,3.0,9.0,10.0,9.0,10.0,10.0,2.0,0.0,1.0,2.0,1.0,10.0,2.0,3.0,3.0,2.0,3.0,4.0,4.0,1.0,1.0,5.0,5.0,3.0,3.0,2.0,3.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,7.0,7.0,7.0,9.0,1.0,1.0,1.0,1.0,10.0,10.0,10.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,3.0,4.0,1.0,2.0,84.0,4,9.0,0.654310,0.442999,0.664499,2012
32657,392,JPN,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,7.0,7.0,5.0,2.0,8.0,1.0,2.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,3.0,1.0,4.0,1.0,3.0,1.0,2.0,4.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,8.0,7.0,6.0,1.0,4.0,8.0,1.0,1.0,1.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,2.0,1.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,3.0,3.0,2.0,8.0,10.0,10.0,9.0,10.0,10.0,6.0,5.0,9.0,10.0,1.0,7.0,1.0,2.0,2.0,10.0,3.0,3.0,3.0,2.0,4.0,4.0,4.0,1.0,1.0,1.0,1.0,4.0,4.0,1.0,1.0,2.0,4.0,4.0,4.0,3.0,9.0,9.0,7.0,7.0,10.0,8.0,1.0,1.0,1.0,1.0,7.0,8.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,2.0,79.0,4,9.0,1.000000,0.409333,0.613999,2010
20338,276,GMY,1.0,3.0,3.0,3.0,4.0,4.0,3.0,4.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,5.0,1.0,7.0,3.0,2.0,3.0,1.0,4.0,1.0,2.0,3.0,2.0,1.0,4.0,5.0,1.0,4.0,5.0,5.0,6.0,4.0,5.0,1.0,1.0,2.0,2.0,3.0,4.0,6.0,5.0,4.0,4.0,6.0,1.0,2.0,2.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,3.0,4.0,3.0,1.0,2.0,3.0,8.0,1.0,10.0,7.0,2.0,7.0,6.0,1.0,10.0,9.0,3.0,8.0,1.0,2.0,2.0,6.0,3.0,2.0,4.0,2.0,4.0,4.0,4.0,1.0,1.0,5.0,5.0,3.0,3.0,3.0,2.0,2.0,4.0,4.0,4.0,4.0,8.0,9.0,3.0,2.0,2.0,7.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,4.0,1.0,2.0,4.0,2.0,93.0,4,5.0,0.441117,0.215600,0.323399,2013
37980,422,LEB,1.0,1.0,2.0,2.0,3.0,1.0,2.0,3.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,5.0,6.0,1.0,6.0,9.0,1.0,2.0,3.0,4.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,9.0,1.0,1.0,10.0,10.0,1.0,1.0,1.0,2.0,3.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,10.0,1.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,9.0,4.0,5.0,1.0,2.0,2.0,10.0,2.0,2.0,2.0,2.0,3.0,4.0,4.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,10.0,10.0,1.0,10.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,10.0,1.0,1.0,10.0,1.0,2.0,2.0,2.0,2.0,1.0,4.0,2.0,4.0,5.0,1.0,65.0,4,8.0,1.000000,0.833333,1.250000,2013
5997,112,BLR,1.0,1.0,1.0,3.0,3.0,2.0,1.0,4.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,1.0,3.0,3.0,1.0,2.0,8.0,10.0,5.0,2.0,7.0,2.0,3.0,1.0,3.0,1.0,4.0,1.0,3.0,1.0,1.0,2.0,5.0,1.0,4.0,1.0,4.0,5.0,1.0,1.0,1.0,5.0,2.0,2.0,2.0,5.0,9.0,1.0,1.0,1.0,7.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,1.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,2.0,5.0,2.0,10.0,7.0,8.0,10.0,5.0,6.0,9.0,10.0,1.0,3.0,1.0,2.0,2.0,8.0,3.0,2.0,3.0,2.0,2.0,3.0,4.0,1.0,1

In [289]:
# how many non-missing age values? (should be 0 since based on imputed missings)
wvs_w6.v242g.notna().sum()

84868

In [291]:
# remove original age variable 'v242'
#wvs_w6.drop(columns = ['v242'], axis = 1, inplace = True)

In [292]:
# export to csv to do feature engineering in a separate notebook
#wvs_w6.to_csv('../data/Evaluating_Happiness/w6_no_missings.csv', index=False)